In [ ]:
import coiled
from dask.distributed import Client
import dask

In [ ]:
import dask.dataframe as dd

In [ ]:
cluster = coiled.Cluster(n_workers=100)
client = cluster.get_client()

In [ ]:
client

In [ ]:
dask.config.set({"dataframe.convert-string": True})

In [ ]:
df = dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/",
    storage_options={'anon': True}
)

In [ ]:
df_full = df

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
dask.utils.format_bytes(
    df.memory_usage(deep=True).sum().compute()
)

In [ ]:
df_sample = df.sample(frac=0.2)

In [ ]:
df_sample = df_sample.persist()

In [ ]:
dask.utils.format_bytes(
    df_sample.memory_usage(deep=True).sum().compute()
)

In [ ]:
from dask.sizeof import sizeof

In [ ]:
partitions_mem_stats = df_sample.map_partitions(sizeof).compute()

In [ ]:
(partitions_mem_stats / 1024**2).describe() #in MiB

In [ ]:
dask.utils.format_bytes(df_sample.partitions[0].memory_usage(deep=True).compute().sum())

In [ ]:
%%time
#passanger fare
df_sample.base_passenger_fare.sum().compute() / 1e9

In [ ]:
%%time
#tip
df_sample.tips.sum().compute() / 1e6

In [ ]:
%%time
df_sample.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

## Partition size 1MB 

Runs are ~11X slower compared to 13MB partitions

In [ ]:
df_sample = df_sample.repartition(partition_size="10MB").persist()

In [ ]:
dask.utils.format_bytes(df_sample.partitions[0].memory_usage(deep=True).compute().sum())

In [ ]:
%%time
#passanger fare
df_sample.base_passenger_fare.sum().compute() / 1e9

In [ ]:
%%time
#tip
df_sample.tips.sum().compute() / 1e6

In [ ]:
%%time
df_sample.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

## Write 1MB partition data to parquet and csv

In [ ]:
df_sample.to_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-0.2-10/");

In [ ]:
df_sample.to_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-0.2-10/");

In [ ]:
ddf = dd.read_csv(
    "s3://coiled-datasets/uber-lyft-tlc-sample/csv-0.2-10/*", 
    dtype={"wav_match_flag": "category"},
)

In [ ]:
dask.utils.format_bytes(
    ddf.memory_usage(deep=True).sum().compute()
)